# [[공빅데] Naver 오픈API를 이용하여 원하는 검색어로 블로그 크롤링 하기(제목+본문)](https://wonhwa.tistory.com/entry/python-Naver-%EC%98%A4%ED%94%88API%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC-%EC%9B%90%ED%95%98%EB%8A%94-%EA%B2%80%EC%83%89%EC%96%B4%EB%A1%9C-%EB%B8%94%EB%A1%9C%EA%B7%B8-%ED%81%AC%EB%A1%A4%EB%A7%81-%ED%95%98%EA%B8%B0%EC%A0%9C%EB%AA%A9%EB%B3%B8%EB%AC%B8?category=996518)

### step1. 네이버 open API를 이용하여 원하는 검색어로 관련 게시물 불러오기

신청한 네이버 open API 아이디와 시크릿 코드를 이용하여 블로그 게시물을 불러온다. 예시로 검색어를 '상암 맛집'으로, 출력 개수는 10개를 받아온다.

참고: [[공빅데] Naver 오픈API를 이용하여 크롤링 하기(뉴스/블로그/카페)](http://localhost:8888/lab/tree/Python_projects/%5B%EA%B3%B5%EB%B9%85%EB%8D%B0%5D%20Naver%20%EC%98%A4%ED%94%88API%EB%A5%BC%20%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC%20%ED%81%AC%EB%A1%A4%EB%A7%81%20%ED%95%98%EA%B8%B0(%EB%89%B4%EC%8A%A4%2C%20%EB%B8%94%EB%A1%9C%EA%B7%B8%2C%20%EC%B9%B4%ED%8E%98).ipynb)

In [28]:
import os
import sys
import urllib.request
import json
client_id = "WC9okFQ1bzGU98jzZuIg" # 발급받은 id 입력
client_secret = "HYGMuu8_cO" # 발급받은 secret 입력 
quote = input("검색어를 입력해주세요.: ") #검색어 입력받기
encText = urllib.parse.quote(quote)
display_num = input("검색 출력결과 갯수를 적어주세요.(최대100, 숫자만 입력): ") #출력할 갯수 입력받기
url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&display="+display_num# json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    #print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

body = response_body.decode('utf-8')
body

검색어를 입력해주세요.:  상암 맛집
검색 출력결과 갯수를 적어주세요.(최대100, 숫자만 입력):  10


'{\n"lastBuildDate": "Sun, 31 Oct 2021 18:30:18 +0900",\n"total": 44029,\n"start": 1,\n"display": 10,\n"items": [\n{\n"title": "감격했던 <b>상암 맛집</b> 리스트",\n"link": "https:\\/\\/blog.naver.com\\/lovableje?Redirect=Log&logNo=222508972195",\n"description": "며칠 전 남편과 같이 유명한 <b>상암 맛집</b>에 방문했는데요 이베리코돼지고기를 부위별로 즐길 수 있었던... 스시토와 점심: 38,000원 / 저녁 90,000원 주소: 서울 마포구 상암동 12-74 전화번호: ***-****-**** 영업시간: 12... ",\n"bloggername": "쯘이의 꽃같은일상",\n"bloggerlink": "https://blog.naver.com/lovableje",\n"postdate": "20210917"\n\n},\n{\n"title": "흡족했던 <b>상암 맛집</b> 리스트",\n"link": "https:\\/\\/blog.naver.com\\/leeih009?Redirect=Log&logNo=222408692869",\n"description": "며칠 전, 직장동료들과 유명한 <b>상암 맛집</b>에서 점심식사를 했는데요. 푸짐한 구성의 스키야키를 빠른시간 내에 맛볼 수 있어서 흡족했던 곳이랍니다. 1. 옥소반 상암점 런치-스키야키 A세트 11,900원 주소... ",\n"bloggername": "Hyun Lab",\n"bloggerlink": "https://blog.naver.com/leeih009",\n"postdate": "20210626"\n\n},\n{\n"title": "무한흡입 가능한 <b>상암 맛집</b>",\n"link": "https:\\/\\/blog.naver.com\\/zhao0201?Redirect=Log&logNo=222019408

위를 보면 블로그 게시물에 대한 정보들이 나와있다.  
여기서 title과 link를 추출하도록 한다.  
여기서는 블로그에 대략적인 내용만 나오지 블로그 내용 전체가 나오지 않으므로  
블로그 링크를 추출하여 각각의 블로그로 들어가 내용을 크롤링 하도록 해야 한다.

### step2. 게시글 제목 및 링크만 추출하기
위의 결괏값을 보면 "(큰 따옴표)가 글 앞뒤로 붙어있는데 replace를 활용해서 지운다.

In [29]:
# 불필요한 ""(큰따옴표)지워주기
body = body.replace('"','')

In [45]:
body

"{\nlastBuildDate: Sun, 31 Oct 2021 18:30:18 +0900,\ntotal: 44029,\nstart: 1,\ndisplay: 10,\nitems: [\n{\ntitle: 감격했던 <b>상암 맛집</b> 리스트,\nlink: https:\\/\\/blog.naver.com\\/lovableje?Redirect=Log&logNo=222508972195,\ndescription: 며칠 전 남편과 같이 유명한 <b>상암 맛집</b>에 방문했는데요 이베리코돼지고기를 부위별로 즐길 수 있었던... 스시토와 점심: 38,000원 / 저녁 90,000원 주소: 서울 마포구 상암동 12-74 전화번호: ***-****-**** 영업시간: 12... ,\nbloggername: 쯘이의 꽃같은일상,\nbloggerlink: https://blog.naver.com/lovableje,\npostdate: 20210917\n\n},\n{\ntitle: 흡족했던 <b>상암 맛집</b> 리스트,\nlink: https:\\/\\/blog.naver.com\\/leeih009?Redirect=Log&logNo=222408692869,\ndescription: 며칠 전, 직장동료들과 유명한 <b>상암 맛집</b>에서 점심식사를 했는데요. 푸짐한 구성의 스키야키를 빠른시간 내에 맛볼 수 있어서 흡족했던 곳이랍니다. 1. 옥소반 상암점 런치-스키야키 A세트 11,900원 주소... ,\nbloggername: Hyun Lab,\nbloggerlink: https://blog.naver.com/leeih009,\npostdate: 20210626\n\n},\n{\ntitle: 무한흡입 가능한 <b>상암 맛집</b>,\nlink: https:\\/\\/blog.naver.com\\/zhao0201?Redirect=Log&logNo=222019408054,\ndescription: 며칠 전 친구들과 초밥이 먹고 싶어서 <b>상암 맛집</b>에 찾아갔다. 무한리필집이라

그 다음 "가 없어진 내용을 가지고 제목을 추출한다.  
제목은 **title:** 뒤, **link** 뒤에 위치하므로 re를 사용해서 제목만 추출한다.

In [30]:
#블로그 제목만 뽑기
import re
titles = re.findall('title: (.*?),\nlink', body) # \n은 엔터의 의미
print('<<제목 모음>>')
print(titles)
print('총 제목 수: ',len(titles),'개') # 제목 갯수 확인

<<제목 모음>>
['감격했던 <b>상암 맛집</b> 리스트', '흡족했던 <b>상암 맛집</b> 리스트', '무한흡입 가능한 <b>상암 맛집</b>', '행복이 가득한 <b>상암 맛집</b> 모음집', '경이로운 <b>상암 맛집</b> 모음', '쫄깃했던 <b>상암 맛집</b>', '감탄이 절로 나오는 <b>상암 맛집</b>', '훌륭했던 <b>상암 맛집</b> 리스트', '29,000 코스요리 만족! <b>상암</b> 파스타<b>맛집</b> 인칸토 키친', '<b>상암 맛집</b> 후라토식당! 규카츠가 맛있는 집']
총 제목 수:  10 개


In [32]:
# 블로그 링크들 추출
links = re.findall('link: (.*?),\ndescription', body)
print('\n<<링크 모음>>')
print(links)
print('총 링크 수: ',len(links),'개') # 링크 갯수 확인


<<링크 모음>>
['https:\\/\\/blog.naver.com\\/lovableje?Redirect=Log&logNo=222508972195', 'https:\\/\\/blog.naver.com\\/leeih009?Redirect=Log&logNo=222408692869', 'https:\\/\\/blog.naver.com\\/zhao0201?Redirect=Log&logNo=222019408054', 'https:\\/\\/blog.naver.com\\/natory77?Redirect=Log&logNo=222098984062', 'https:\\/\\/blog.naver.com\\/hyuni9987?Redirect=Log&logNo=222217005651', 'https:\\/\\/blog.naver.com\\/brightenedu?Redirect=Log&logNo=221766379557', 'https:\\/\\/blog.naver.com\\/daebakyo?Redirect=Log&logNo=222154100950', 'https:\\/\\/blog.naver.com\\/rostw92?Redirect=Log&logNo=222425036028', 'https:\\/\\/blog.naver.com\\/wise9140?Redirect=Log&logNo=222531769606', 'https:\\/\\/blog.naver.com\\/rdal89?Redirect=Log&logNo=222496358947']
총 링크 수:  10 개


위의 링크를 확인해 보면 보통의 url과는 다르게 '\\'(역 슬래쉬)가 많이 들어가 있음을 확인할 수 있다.  
보통의 url링크를 가져오기 위해 링크 사이의 '\\'를 지워 다듬어진 링크를 가져오도록 하겠다.

In [33]:
# 링크를 다듬기 (필요없는 부분 제거 및 수정)
blog_links = []
for i in links:
    a = i.replace('\\', '')
    b = a.replace('?Redirect=Log&logNo=', '/')
    blog_links.append(b)
    
print(blog_links)
print('생성된 링크 개숫: ', len(blog_links), '개')

['https://blog.naver.com/lovableje/222508972195', 'https://blog.naver.com/leeih009/222408692869', 'https://blog.naver.com/zhao0201/222019408054', 'https://blog.naver.com/natory77/222098984062', 'https://blog.naver.com/hyuni9987/222217005651', 'https://blog.naver.com/brightenedu/221766379557', 'https://blog.naver.com/daebakyo/222154100950', 'https://blog.naver.com/rostw92/222425036028', 'https://blog.naver.com/wise9140/222531769606', 'https://blog.naver.com/rdal89/222496358947']
생성된 링크 개숫:  10 개


### step3. 게시글 본문 가져오기
다음은 블로그 링크를 활용하여 각 페이지에 접근해 본문을 가져온다.  
naver 블로그는 각 게시물에 들어가 F12를 눌러 보면 아래와 같이 iframe 안에 게시물 본문 글이 위치한다.

때문에 Selenium을 이용하여 각 게시물의 iframe 접근 후 내용을 추출하는 방식으로 크롤링을 해야 한다.  
Selenium을 이용할 때는 전용 drvier 가 있어야 한다.  
여기서는 크롬 드라이버를 이용한다. [여기](https://sites.google.com/a/chromium.org/chromedriver/downloads)를 클릭하여 현재 본인이 사용하고 있는 구글 버전에 맞는 드라이버를 다운받는다.

In [38]:
# 본문 크롤링
import time
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(3)

그다음으로 아까 다듬어진 링크에 접속하여 iframe 접근 후 본문의 내용을 크롤링 한다.  
본문은 **iframe** 안의 **<dim> class='se-main-container'**에 있음을 확인할 수 있다.

In [40]:
# 블로그 링크 하나씩 불러서 크롤링
contents = []
for i in blog_links:
    # 블로그 링크 하나씩 불러오기
    driver.get(i)
    time.sleep(1)
    # 블로그 안 본문이 있는 iframe에 접근하기
    driver.switch_to.frame("mainFrame")
    # 본문 내용 크롤링하기
    a = driver.find_element_by_css_selector('div.se-main-container').text
    contents.append(a)
    # print(본문: '\n', a)
    
driver.quit()
print('<<본문 크롤링이 완료되었습니다.>>')
contents

<<본문 크롤링이 완료되었습니다.>>


['며칠 전 남편과 같이 유명한 상암 맛집에 방문했는데요 \n이베리코돼지고기를 부위별로 즐길 수 있었던 곳이라 여러분들에게 소개해드려요  \n1. 이베리코민족\n플루마: 18,000원 / 황제살 22,000원\n주소: 서울 마포구 월드컵북로 345 1층\n전화번호: 02-304-4353\n영업시간: 11:30~22:00 (일요일 휴무)\n50m\n© NAVER Corp.\n이베리코민족 상암점\n서울특별시 마포구 월드컵북로 345 1층\n이 날 방문한 이베리코민족은 디지털미디어시티역에서 차로 2분정도 소요됐어요 \n밝게 빛나는 조명 간판 덕분에 처음 방문하는 것이었지만 쉽게 찾을 수 있었죠 \n바로 앞 주차 공간에 주차를 하고 안으로 들어서려는데 배너가 있더라고요 \n점심메뉴를 상암 맛집에서 판매하고 있는지라 시간이 되면 점심에도 와보고싶었어요  \n바로 안으로 들어서자 QR코드와 발열체크기가 세워져 있었어요.\n손소독까지 마쳐야 안으로 들어설 수 있도록 안내해주셔서 괜히 안심되더라고요 \n내부 홀은 톤다운된 분위기를 띄고 있었고 조명이 은은하게 비추고 있었어요 \n자리마다 깔끔하게 치워진 모습에 기분 좋게 입장할 수 있었답니다. \n벽면에는 원산지 표시가 되어 있었는데요 \n여러가지 식재료별 그리고 음식명으로도 기입을 해놓아서 명확하게 알 수 있었어요.\n앉고싶은 자리를 초이스하고 난 뒤 상암 맛집의 메뉴판을 스캔해보았어요.\n플루마, 프레샤 등 조금 생소한 부위였지만 직원분께서 친절하게 설명해주신 덕에 잘 고를 수 있었답니다. \n주문을 마치자 개별 생수병과 일회용컵이 내어졌어요.\n아무래도 시기가 이런 만큼 개인별로 내어준 모습에 은근히 안심이 되더라고요 \n조금 시간이 흐르고 밑반찬들을 하나씩 테이블 위로 올려주셨어요.\n모두 고기와 함께 곁들였을 때 조합이 좋은 구성이라서 식욕이 폭발하더라고요 \n먼저 달콤한 맛이 일품인 고구마샐러드 한스쿱을 맛보았는데요.\n소담하게 담겨진 모습이 플레이팅까지 신경쓴 것 같아서 센스있게 느껴졌네요.\n이어서 뽀얀 빛깔을

이렇게 모든 게시물의 본문 내용을 가져올 수 있다.

### step4. 크롤링 내용들 DataFrame으로 만들기

마지막으로 아까 추출한 제목, 블로그, 링크 내용을 DataFrame으로 만들어 깔끔하게 정리한다.

In [44]:
import pandas as pd
df = pd.DataFrame({'제목': titles,
                  '링크': blog_links,
                  '내용': contents})
df

,제목,링크,내용
0,감격했던 <b>상암 맛집</b> 리스트,https://blog.naver.com/lovableje/222508972195,며칠 전 남편과 같이 유명한 상암 맛집에 방문했는데요 \n이베리코돼지고기를 부위별로...
1,흡족했던 <b>상암 맛집</b> 리스트,https://blog.naver.com/leeih009/222408692869,"며칠 전, 직장동료들과 유명한 상암 맛집에서 점심식사를 했는데요. 푸짐한 구성의 스..."
2,무한흡입 가능한 <b>상암 맛집</b>,https://blog.naver.com/zhao0201/222019408054,며칠 전 친구들과 초밥이 먹고 싶어서 상암 맛집에 찾아갔다.\n무한리필집이라 배불리...
3,행복이 가득한 <b>상암 맛집</b> 모음집,https://blog.naver.com/natory77/222098984062,1. 인기명 상암점\n50m\n© NAVER Corp.\n인기명 상암점\n서울특별...
4,경이로운 <b>상암 맛집</b> 모음,https://blog.naver.com/hyuni9987/222217005651,오랜만에 마포에 갔다가 점심 먹으러 상암 맛집에 다녀왔어요.\n한약재로 우려낸 조갈...
5,쫄깃했던 <b>상암 맛집</b>,https://blog.naver.com/brightenedu/221766379557,"Hello, everyone!\n김원장의 탐구생활입니다. \n끝나지 않을 것만 같은..."
6,감탄이 절로 나오는 <b>상암 맛집</b>,https://blog.naver.com/daebakyo/222154100950,하이 하이 괴짜 미녀임>.<ㅋㅋㅋ 며칠 전에 친구들이랑 다녀온 상암 맛집에서 진짜 ...
7,훌륭했던 <b>상암 맛집</b> 리스트,https://blog.naver.com/rostw92/222425036028,저번 주 토요일에 친한 언니와 함께 집 근처인 상암동에 다녀왔다.\n저녁도 먹기로 ...
8,"29,000 코스요리 만족! <b>상암</b> 파스타<b>맛집</b> 인칸토 키친",https://blog.naver.com/wise9140/222531769606,#가성비코스요리 #인칸토키친 #상암맛집\n안녕하세요 여러분 수냥입니다 :)\n맛집 ...
9,<b>상암 맛집</b> 후라토식당! 규카츠가 맛있는 집,https://blog.naver.com/rdal89/222496358947,"상암 맛집 후라토식당, 하필 왜 여기를?\n근처에 있는 하늘공원에 다녀온 뒤, 디지..."
